<a href="https://colab.research.google.com/github/walnashgit/S30-Capstone/blob/main/S30ProjectionLayerTrainingipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
#!/bin/bash
!kaggle datasets download walnash/coco2014embeddingsm2


Dataset URL: https://www.kaggle.com/datasets/walnash/coco2014embeddingsm2
License(s): unknown
100% 6.64G/6.65G [01:30<00:00, 124MB/s]
100% 6.65G/6.65G [01:30<00:00, 78.5MB/s]


In [ ]:
!unzip /content/coco2014embeddingsm2.zip

Archive:  /content/coco2014embeddingsm2.zip
  inflating: captions_train2014.json  
  inflating: coco2014_clip_embeddings_m2.h5  


In [ ]:
cfg = dict(
    num_epoch=1,
    last_epoch=2,
    saved_model='', # model name with dir
    resume=True,
    data_dir='../data',
    checkpoint_dir='./checkpoint',
    max_seqlen=80,
    batch_size=2,
    vision_projector_file='',
    validation_phase=False,
    clip_dim=768, #512,
    phi_dim=2560,
    image_token="<image>",
)
IMAGE_TOKEN_INDEX = -200

In [ ]:
def tokenizer_image_token(prompt, tokenizer, image_token_index=IMAGE_TOKEN_INDEX, return_tensors=None):
    prompt_chunks = [tokenizer(chunk).input_ids for chunk in prompt.split('<image>')]

    def insert_separator(X, sep):
        return [ele for sublist in zip(X, [sep]*len(X)) for ele in sublist][:-1]

    input_ids = []
    offset = 0
    if len(prompt_chunks) > 0 and len(prompt_chunks[0]) > 0 and prompt_chunks[0][0] == tokenizer.bos_token_id:
        offset = 1
        input_ids.append(prompt_chunks[0][0])
    for x in insert_separator(prompt_chunks, [image_token_index] * (offset + 1)):
        input_ids.extend(x[offset:])

    if return_tensors is not None:
        if return_tensors == 'pt':
            return torch.tensor(input_ids, dtype=torch.long)
        raise ValueError(f'Unsupported tensor type: {return_tensors}')
    return input_ids

In [ ]:
import json
import random

import torch
from torch.utils.data import Dataset, Sampler
import h5py


class ProjectionLayerDataset2(Dataset):
    def __init__(self, embedding_file, caption_file, tokenizer, max_length=80):
        self.embedding_file = h5py.File(embedding_file, 'r')
        self.image_ids = set(self.embedding_file.keys())
        self.caption_file = caption_file
        self.captions = self.get_caption_dict()  # List of text inputs
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.image_token = cfg['image_token']

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        caption = self.captions[idx]
        image_id = str(caption.get('image_id'))
        image_embedding = torch.tensor(self.embedding_file[image_id][()])
        image_embedding = image_embedding[1:, :]  # returning 49 x 768
        text = caption.get('caption')
        prompt = f'{self.image_token} caption: '

        prompt_ids = torch.tensor(tokenizer_image_token(prompt, tokenizer=self.tokenizer), dtype=torch.int32)
        # prompt_ids = tokenized_prompt['input_ids']
        labels = self.tokenizer.encode(text)
        labels = self.get_final_label_ids(labels, prompt_ids, image_embedding)

        return {
            'image_embedding': image_embedding,
            'prompt_ids': prompt_ids, #torch.tensor(prompt_ids, dtype=torch.int32),
            'labels': labels,
            # 'attention_mask': torch.tensor(tokenized['attention_mask']),
            # 'image_token_position': image_token_position
        }

    def get_final_label_ids(self, labels, prompt_ids, image_embedding):
        pad_token_count = self.max_length - (prompt_ids.size(0) + image_embedding.size(0) - 1) - len(labels) - 1
        if pad_token_count < 0:
            pad_token_count = 0
            # truncate_len = self.max_length - (len(token_ids) + prompt_ids.size(0) - 1) - 1
            truncate_len = self.max_length - (prompt_ids.size(0) + image_embedding.size(0) - 1) - 1
            labels = labels[:truncate_len]

        labels = torch.cat(
            [
                torch.tensor(labels, dtype=torch.int32),
                torch.tensor([self.tokenizer.eos_token_id], dtype=torch.int32),
                torch.tensor([self.tokenizer.pad_token_id] * pad_token_count, dtype=torch.int32)
            ],
            dim=0
        )
        return labels


    def get_caption_dict(self):
        with open(self.caption_file, 'r') as f:
            caption_file_json = json.load(f)
            # return [cap for cap in caption_file_json['annotations']]
            return [
                cap for cap in caption_file_json['annotations']
                if str(cap['image_id']) in self.image_ids
            ]

In [ ]:
import torch

class ProjectionPreTrainer(torch.nn.Module):
    def __init__(self, projectionModel, phi_model, phi_tokenizer, device='cuda'):
        super().__init__()
        self.device = device
        self.projectionModel = projectionModel
        self.phi_model = phi_model
        self.phi_tokenizer = phi_tokenizer
        self.phi_model.to(device)
        self.phi_embeddings = self.phi_model.get_input_embeddings()
        self.loss = nn.CrossEntropyLoss()

    def forward(self, image_embedding, label_ids, prompt_ids):
        # Get Phi-2 text embeddings
        projected_clip = self.projectionModel(image_embedding)
        # with torch.no_grad():
        new_prompt_embeds = self.prepare_input_embed(projected_clip, prompt_ids)
        ie_size = new_prompt_embeds.size(1) - 1

        phi_text_embedding = self.phi_embeddings(label_ids)
        combined_embeddings = torch.cat(
            [
                new_prompt_embeds,
                phi_text_embedding
            ],
            dim=1
        )
        phi_outputs = self.phi_model(inputs_embeds=combined_embeddings)

        logits = phi_outputs['logits']

        X = logits[:, ie_size:ie_size + label_ids.size(1), :]
        Y = label_ids.contiguous().type(torch.LongTensor).to(self.device)

        X = X.contiguous().view(-1, X.size(-1))
        Y = Y.view(-1)

        loss_val = self.loss(
            X,
            Y
        )

        return logits, loss_val

    def prepare_input_embed(self, image_embeds, prompt_ids):
        new_input_embeds = []
        for batch_idx, cur_prompt_ids in enumerate(prompt_ids):
            image_token_indices = torch.where(cur_prompt_ids == IMAGE_TOKEN_INDEX)[0]
            cur_new_input_embeds = []

            image_token_start = image_token_indices[0]

            cur_new_input_embeds.append(self.phi_embeddings(cur_prompt_ids[:image_token_start]))
            cur_new_input_embeds.append(image_embeds[batch_idx])
            cur_new_input_embeds.append(self.phi_embeddings(cur_prompt_ids[image_token_start + 1:]))
            cur_new_input_embeds = [x.to(self.device) for x in cur_new_input_embeds]
            cur_new_input_embeds = torch.cat(cur_new_input_embeds, dim=0)

            new_input_embeds.append(cur_new_input_embeds)

        new_input_embeds = torch.stack(new_input_embeds, dim=0)

        return new_input_embeds

In [ ]:
class ProjectionLayer(torch.nn.Module):
    def __init__(self, clip_dim, phi_dim):
        super().__init__()
        self.linear = torch.nn.Linear(clip_dim, phi_dim)

    def forward(self, x):
        return self.linear(x)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import gc


phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
phi_tokenizer.pad_token = phi_tokenizer.eos_token


def pretrain_projection(model, dataloader, device):
    model = model.to(device)
    # optimizer = torch.optim.Adam(model.projectionModel.parameters(), lr=0.001)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    # criterion = CosineSimilarityLoss()
    # criterion = nn.CrossEntropyLoss()

    last_epoch = 0
    prev_loss = 10000
    if cfg['resume']:
        saved_model = torch.load(cfg['saved_model'])
        model.projectionModel.load_state_dict(saved_model['model_state_dict'])
        optimizer.load_state_dict(saved_model['optimizer_state_dict'])
        last_epoch = saved_model['last_epoch']
        prev_loss = saved_model['loss']
        print('prev_loss: ', prev_loss)
        print('last_epoch: ', last_epoch)

    step_count = -1
    num_epochs = cfg['num_epoch']
    # model.projectionModel.train()

    bestLoss = 0
    bestStep = 0

    for epoch in range(last_epoch, num_epochs):
        total_loss = 0
        step_loss = 0
        # for batch in dataloader:
        batch_iterator = tqdm(dataloader, desc=f"Processing Epoch {epoch:02d}")
        # for batch in itertools.islice(batch_iterator, 20):
        for batch in batch_iterator:
            optimizer.zero_grad()
            clip_embedding = batch['image_embedding']

            label_ids = batch['labels']
            prompt_ids = batch['prompt_ids']

            clip_embedding = clip_embedding.to(device).requires_grad_(True)
            label_ids = label_ids.to(device)
            prompt_ids = prompt_ids.to(device)

            with torch.autocast(device_type=device, dtype=torch.float16):

              logits, loss = model(clip_embedding, label_ids, prompt_ids)

              total_loss += loss.item()
              step_loss += loss.item()

              loss.backward()

            if step_count == -1:
              print(f"\n Epoch {epoch + 1}, step: {step_count}, loss: {loss.item()}, total loss: {total_loss}")

            if loss.item() < prev_loss:
                bestLoss = loss.item()
                bestStep = step_count
                print(f"\n Epoch {epoch + 1}, step: {step_count}, Loss: {loss.item()}, total loss: {total_loss}")
                save_model(epoch, model, loss.item(), optimizer, step_count)
                # print('saving model')
                prev_loss = loss.item()
            elif step_count > 0 and step_count % 100 == 0:
                print(f"\n Epoch {epoch + 1}, step: {step_count}, loss: {loss.item()}, total loss: {total_loss}")

            step_count += 1
            optimizer.step()

            gc.collect()
            torch.cuda.empty_cache()

        if device == 'cuda':
            torch.cuda.empty_cache()
        elif device == 'mps':
            torch.mps.empty_cache()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")
        print(f"BestLoss: {bestLoss}, BestStep: {bestStep}")


def save_model(epoch, model, loss, optimizer, step_count):
    print("saving model")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.projectionModel.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'step_count': step_count,
    }, '%s/projectionModel_ckpt_%s.pth' % (cfg['checkpoint_dir'], epoch))


def train():
    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    elif torch.backends.mps.is_available():
        device = 'mps'

    gc.collect()
    torch.cuda.empty_cache()

    torch.manual_seed(14)


    embed_file = '/content/coco2014_clip_embeddings_m2.h5'
    caption_file = '/content/captions_train2014.json'
    dataset = ProjectionLayerDataset2(embed_file, caption_file, phi_tokenizer)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True, pin_memory=False)


    model = get_model(device)
    pretrain_projection(model, dataloader, device)


def get_model(device):
    projectionModel = ProjectionLayer(cfg['clip_dim'], cfg['phi_dim'])
    phi_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", trust_remote_code=True, torch_dtype=torch.float16, low_cpu_mem_usage=True)

    projectionModel.train()
    phi_model.eval()  # Set Phi-2 to evaluation mode
    for param in phi_model.parameters():
        param.requires_grad = False

    return ProjectionPreTrainer(projectionModel, phi_model, phi_tokenizer, device)


def find_image_toke_pos(self, input_ids):
    # Find the position of the <image> token
    image_token = cfg['image_token']
    image_token_id = self.phi_tokenizer.convert_tokens_to_ids(image_token)
    image_token_position = (torch.tensor(input_ids) == image_token_id).nonzero(as_tuple=True)[0]
    return image_token_position

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Epoch 00:   0%|          | 0/15633 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)



 Epoch 1, step: -1, Loss: 6.3405938148498535, total loss: 6.3405938148498535
saving model


Processing Epoch 00:   0%|          | 23/15633 [00:32<6:00:24,  1.39s/it]


 Epoch 1, step: 22, Loss: 6.311634063720703, total loss: 160.87029933929443
saving model


...


 Epoch 1, step: 6300, loss: 1.663983702659607, total loss: 12670.272683501244


Processing Epoch 00:  41%|████      | 6401/15633 [2:31:40<3:40:25,  1.43s/it]


 Epoch 1, step: 6400, loss: 1.6569445133209229, total loss: 12832.472758054733


Processing Epoch 00:  41%|████      | 6405/15633 [2:31:46<3:38:39,  1.42s/it]


KeyboardInterrupt: 

# Multimodal GPT - Trained on PHI2 LLM

This project is a multimodal GPT capable of handling images, audio and text as input for context for question and answer from the GPT. This uses PHI2 LLM.

## Training

### Pre training
To handle image as input, a small projection layer was pre-trained which was used to project the input image embedding to the same latent space that PHI2 can handle.


* Image embedding is obtained using CLIP vision model.
* Image embedding is passed through projecting layer.
* Projected image embedding is concatenated with the embedding of the corresponding caption embedding got from the PHI2 text embedding. For this step the parameters of PHI2 is frozen so that it does change during back propagation.
* The concatenated embedding is used as input to PHI2 model and is compared with actual caption for loss.
* Datset used is [COCO2014](https://www.kaggle.com/datasets/nadaibrahim/coco2014).


```
class ProjectionLayer(torch.nn.Module):
    def __init__(self, clip_dim, phi_dim):
        super().__init__()
        self.linear = torch.nn.Linear(clip_dim, phi_dim)

    def forward(self, x):
        return self.linear(x)
```

<Image>

### Pre-Training Log

```
 Epoch 1, step: 4000, loss: 1.7207248210906982, total loss: 8806.965348124504
Processing Epoch 00:  26%|██▌       | 4101/15633 [1:37:46<4:29:48,  1.40s/it]

 Epoch 1, step: 4100, loss: 1.7700507640838623, total loss: 8986.26793396473
Processing Epoch 00:  27%|██▋       | 4201/15633 [1:40:06<4:23:33,  1.38s/it]

 Epoch 1, step: 4200, loss: 1.7114845514297485, total loss: 9165.797843456268
Processing Epoch 00:  28%|██▊       | 4301/15633 [1:42:28<4:22:25,  1.39s/it]

 Epoch 1, step: 4300, loss: 1.8178526163101196, total loss: 9349.606652259827
Processing Epoch 00:  28%|██▊       | 4401/15633 [1:44:48<4:29:18,  1.44s/it]

 Epoch 1, step: 4400, loss: 1.713474988937378, total loss: 9528.485793590546
Processing Epoch 00:  29%|██▉       | 4501/15633 [1:47:09<4:23:34,  1.42s/it]

 Epoch 1, step: 4500, loss: 1.8185430765151978, total loss: 9709.111825466156
Processing Epoch 00:  29%|██▉       | 4563/15633 [1:48:37<4:17:50,  1.40s/it]

 Epoch 1, step: 4562, Loss: 1.4059160947799683, total loss: 9814.87218272686
saving model
Processing Epoch 00:  29%|██▉       | 4602/15633 [1:49:31<4:15:43,  1.39s/it]

 Epoch 1, step: 4600, loss: 1.7384731769561768, total loss: 9877.717487335205
Processing Epoch 00:  30%|███       | 4701/15633 [1:51:51<4:15:05,  1.40s/it]

 Epoch 1, step: 4700, loss: 1.7126386165618896, total loss: 10048.258151650429
Processing Epoch 00:  31%|███       | 4777/15633 [1:53:38<4:23:39,  1.46s/it]

 Epoch 1, step: 4776, Loss: 1.3383914232254028, total loss: 10178.873227715492
saving model
Processing Epoch 00:  31%|███       | 4801/15633 [1:54:12<4:19:27,  1.44s/it]

 Epoch 1, step: 4800, loss: 1.7428756952285767, total loss: 10219.085227370262
Processing Epoch 00:  31%|███       | 4884/15633 [1:56:09<4:10:17,  1.40s/it]

 Epoch 1, step: 4883, Loss: 1.3373826742172241, total loss: 10357.566935658455
saving model
Processing Epoch 00:  31%|███▏      | 4901/15633 [1:56:33<4:16:20,  1.43s/it]

 Epoch 1, step: 4900, loss: 1.9052549600601196, total loss: 10385.680579423904
Processing Epoch 00:  32%|███▏      | 5001/15633 [1:58:53<4:06:28,  1.39s/it]

 Epoch 1, step: 5000, loss: 1.8309067487716675, total loss: 10549.668045520782
Processing Epoch 00:  32%|███▏      | 5026/15633 [1:59:29<4:11:36,  1.42s/it]

 Epoch 1, step: 5025, Loss: 1.2582365274429321, total loss: 10589.988594651222
saving model
Processing Epoch 00:  32%|███▏      | 5052/15633 [2:00:05<4:11:53,  1.43s/it]

 Epoch 1, step: 5051, Loss: 1.2571699619293213, total loss: 10630.821589589119
saving model
```



### Fine Tuning
After the projection layer was pretrained, PHI2 was fine tuned.

* Input image embedding is obtained using CLIP vision model.
* Image embedding is passed through pre-trained projecting layer. For fine tuning, the parameters of projection layer is forzen to prevent changes during back propagation.
* Projected embedding is concatenated with the embeddings of Question-Answer text data from [LLaVA-Instruct-150K](https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K)
* Concatenated embedding is used as input to PHI2 model and the output is compared with the answer part for the loss.

<Image>

### Fine Tuning Log
```
 Epoch 1, step: 8580, loss: 2.147397041320801, total loss: 20604.322937846184
clearing cache
Processing Epoch 00:  47%|████▋     | 8591/18148 [7:30:17<8:25:17,  3.17s/it]

 Epoch 1, step: 8590, loss: 2.0401413440704346, total loss: 20627.469371914864
clearing cache
Processing Epoch 00:  47%|████▋     | 8601/18148 [7:30:48<8:24:41,  3.17s/it]

 Epoch 1, step: 8600, loss: 2.2512617111206055, total loss: 20651.634582281113
clearing cache
Processing Epoch 00:  47%|████▋     | 8611/18148 [7:31:19<8:25:37,  3.18s/it]

 Epoch 1, step: 8610, loss: 2.054892063140869, total loss: 20674.192289114
clearing cache
Processing Epoch 00:  48%|████▊     | 8621/18148 [7:31:50<8:22:25,  3.16s/it]

 Epoch 1, step: 8620, loss: 2.16609787940979, total loss: 20694.78110229969
clearing cache
Processing Epoch 00:  48%|████▊     | 8631/18148 [7:32:21<8:22:07,  3.17s/it]

 Epoch 1, step: 8630, loss: 2.1779747009277344, total loss: 20719.170233249664
clearing cache
Processing Epoch 00:  48%|████▊     | 8641/18148 [7:32:52<8:24:29,  3.18s/it]

 Epoch 1, step: 8640, loss: 2.852278709411621, total loss: 20743.28934788704
clearing cache
Processing Epoch 00:  48%|████▊     | 8651/18148 [7:33:23<8:22:59,  3.18s/it]

 Epoch 1, step: 8650, loss: 2.776125431060791, total loss: 20767.941595435143
clearing cache
Processing Epoch 00:  48%|████▊     | 8661/18148 [7:33:54<8:21:47,  3.17s/it]

 Epoch 1, step: 8660, loss: 2.1857335567474365, total loss: 20790.802763819695
clearing cache

```


## Further Improvements
* Fine tuned model is not accurate and can be trained further by improving the data input and loss calculation.
* The context of the queries is limited to the current query and can be improved to include the previous query and responses as context.

